# 파일형식

파일을 나타내는 여러가지 방법이 있습니다. 이에 따라 저장되는 형식, 확장자또한 달라집니다. 대표적인 데이터들에 대해서 알아보겠습니다.

### csv

`comma-separated values` 의 약어로 직역하면 콤마(,)로 불리된 값들입니다. 실제로 파일이 저장될때 각 열의 값들이 콤마로 불리되어 저장됩니다. 예를 들어 이름과 나이를 관리하는 데이터셋일 때 아래와 같습니다.

name, age  
kim, 12  
lee, 15  
park, 11  

실제 데이터를 확인해보겠습니다.

In [ ]:
# caution! 아래 코드는 colab 에서만 작동합니다.
from os.path import join

filename = join("sample_data", "california_housing_test.csv")
values = open(filename).read()

print(values)

### excel

확장자명은 `xlsx` 또는 `xls` 로 저장되며 마이크로소프트 오피스 프로그램 엑셀에서 사용되는 파일 형식입니다. 파일 내용은 엑셀 프로그램을 통해서 여는 것이 아니라면 알아볼 수 없는 문자열들로 이루어져 있습니다. 그 문자열들을 파이썬에서 해석하기 위해서는 `openpyxl` 이나 `pandas` 를 사용해야 합니다.

In [3]:
# caution! 아래 코드는 colab 에서만 작동합니다.
from google.colab import files

uploaded = files.upload()

for filename in uploaded:
    content = uploaded[filename]
    print(content)

# 대용량 데이터에서의 파일형식

## 파일형식이 중요한 이유
- csv, excel 처럼 익숙한 파일형식을 사용할 수 있지만
- 각기 다른 파일형식마다 데이터를 불러오고 저장에 소요되는 시간이 차이납니다
- 또 각각 파일형식에 따라 저장공간을 차지하는 것에 차이를 보입니다

In [4]:
import numpy as np
import pandas as pd

In [5]:
sz = 1_000_000

df = pd.DataFrame({"A": np.random.randn(sz), "B": [1] * sz})

In [6]:
df

,A,B
0,-0.065250,1
1,-1.634458,1
2,-0.206080,1
3,0.464214,1
4,0.463563,1
...,...,...
999995,0.896058,1
999996,-0.561270,1
999997,-1.955262,1
999998,-0.611040,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   A       1000000 non-null  float64
 1   B       1000000 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 15.3 MB


## 출력(파일저장)에 소요되는 시간 측정

In [9]:
%%time

df.to_csv("test.csv")

CPU times: user 1.26 s, sys: 38.8 ms, total: 1.3 s
Wall time: 1.31 s


In [10]:
%%time

df.to_parquet("test.parquet")

CPU times: user 38.4 ms, sys: 17.8 ms, total: 56.3 ms
Wall time: 87.9 ms


## 입력(불러오기)에 소요되는 시간 측정

In [17]:
%%time

_ = pd.read_csv("test.csv")

CPU times: user 135 ms, sys: 19.1 ms, total: 154 ms
Wall time: 153 ms


In [20]:
%%time

_ = pd.read_parquet("test.parquet")

CPU times: user 13.7 ms, sys: 7.92 ms, total: 21.6 ms
Wall time: 15.3 ms


## 저장공간 차이 측정

In [26]:
%ls -alh

total 72288
drwxr-xr-x   7 doyoung  staff   224B Sep 22 02:35 ./
drwxr-xr-x  11 doyoung  staff   352B Sep 20 02:07 ../
drwxr-xr-x   4 doyoung  staff   128B Sep 22 02:27 .ipynb_checkpoints/
-rw-r--r--   1 doyoung  staff    12K Sep 22 01:13 Advanced Python.ipynb
-rw-r--r--   1 doyoung  staff    10K Sep 22 02:35 Untitled.ipynb
-rw-r--r--   1 doyoung  staff    27M Sep 22 02:32 test.csv
-rw-r--r--   1 doyoung  staff   7.9M Sep 22 02:32 test.parquet


# CSV vs Parquet

In [27]:
df = pd.DataFrame({
    "datetime": [pd.Timestamp("2022-01-01"), pd.Timestamp("2022-12-31")],
    "string": ["a", "b"]
})

In [28]:
df.head()

,datetime,string
0,2022-01-01,a
1,2022-12-31,b


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  2 non-null      datetime64[ns]
 1   string    2 non-null      object        
dtypes: datetime64[ns](1), object(1)
memory usage: 160.0+ bytes


## CSV

In [33]:
df.to_csv("test.csv", index=None)

In [34]:
csv_df = pd.read_csv("test.csv")

In [36]:
csv_df.head()

,datetime,string
0,2022-01-01,a
1,2022-12-31,b


In [37]:
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  2 non-null      object
 1   string    2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


## Parquet

In [38]:
df.to_parquet("test.parquet", index=None)

In [39]:
parquet_df = pd.read_parquet("test.parquet")

In [40]:
parquet_df.head()

,datetime,string
0,2022-01-01,a
1,2022-12-31,b


In [41]:
parquet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  2 non-null      datetime64[ns]
 1   string    2 non-null      object        
dtypes: datetime64[ns](1), object(1)
memory usage: 160.0+ bytes


# `sys.path`

In [42]:
import sys

sys.path

['/Users/doyoung/Projects/kepco-class/load-interpolation/problem',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python310.zip',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10/site-packages']

In [43]:
sys.path.append("src")

In [44]:
sys.path

['/Users/doyoung/Projects/kepco-class/load-interpolation/problem',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python310.zip',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniforge/base/envs/load-interpolation/lib/python3.10/site-packages',
 'src']

# 연습문제

`handle_huge_data.csv` 파일을 읽어서 `handle_huge_data_with_parquet.parquet` 파일로 저장해보세요

In [48]:
# 세팅코드입니다
# 수정하지 않고 한번 실행해주세요

import numpy as np
import pandas as pd

huge_data_df = pd.DataFrame({"data": np.random.randn(1_000_000)})
huge_data_df.to_csv("handle_huge_data.csv")

In [ ]:
# 아래에 정답코드를 작성해주세요

